In [0]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  6 15:47:29 2019

@author: Tripti Santani
"""

import numpy as np
import pandas as pd

import logging
logging.getLogger().setLevel(logging.ERROR)


#class feature_builder():
    
def numeric_conversion(listings):

    #Converting into numeric data
    listings['bedrooms'] = pd.to_numeric(listings['bedrooms'],errors='coerce')
    listings['accommodates'] = pd.to_numeric(listings['accommodates'],errors='coerce')
    listings['bathrooms'] = pd.to_numeric(listings['bathrooms'],errors='coerce')
    listings['number_of_reviews']=pd.to_numeric(listings['number_of_reviews'],errors='coerce')
    listings['beds']=pd.to_numeric(listings['beds'],errors='coerce')
    listings['number_of_reviews']=pd.to_numeric(listings['number_of_reviews'],errors='coerce')
    listings['host_listings_count']=pd.to_numeric(listings['host_listings_count'],errors='coerce')
    listings['review_scores_rating']=pd.to_numeric(listings['review_scores_rating'],errors='coerce')
    listings['zipcode']=pd.to_numeric(listings['zipcode'],errors='coerce')
    listings['monthly_price'] = pd.to_numeric(listings['monthly_price'].apply(lambda x: str(x).replace('$', '').replace(',', '')),errors='coerce')
    listings['price'] = pd.to_numeric(listings['price'].apply(lambda x: str(x).replace('$', '').replace(',', '')),errors='coerce')
    listings['weekly_price'] = pd.to_numeric(listings['weekly_price'].apply(lambda x: str(x).replace('$', '').replace(',', '')),errors='coerce')
    listings['security_deposit'] = pd.to_numeric(listings['security_deposit'].apply(lambda x: str(x).replace('$', '').replace(',', '')),errors='coerce')
    listings['cleaning_fee'] = pd.to_numeric(listings['cleaning_fee'].apply(lambda x: str(x).replace('$', '').replace(',', '')),errors='coerce')
    listings['extra_people'] = pd.to_numeric(listings['extra_people'].apply(lambda x: str(x).replace('$', '').replace(',', '')),errors='coerce')
    listings['host_response_rate'] = pd.to_numeric(listings['host_response_rate'].apply(lambda x: str(x).replace('%', '')),errors='coerce')

    df = listings[["host_response_rate", "host_acceptance_rate", "host_is_superhost","security_deposit","cleaning_fee","extra_people",
                   "host_listings_count", "zipcode", "property_type","room_type", "accommodates", "bathrooms", "bedrooms",
                   "beds", "price", "number_of_reviews", "review_scores_rating", "cancellation_policy", 
                   "reviews_per_month"]]
    return df

def missing_value(df):
  
    # Handling the missing values:
    df.price=df.price.fillna(df.price.mean)
    df.security_deposit=df.security_deposit.fillna(0)
    df.reviews_per_month=df.reviews_per_month.fillna(0)
    df.cleaning_fee=df.cleaning_fee.fillna(0)

    # drop NaN rows
    df2=df.dropna(axis=0)
    
    return df2

def feature(df2):

    pd.options.mode.chained_assignment = None  # default='warn'
    df2['host_response_rate'] = df2['host_response_rate'].astype(str)
    df2['host_acceptance_rate'] = df2['host_acceptance_rate'].astype(str)
    df2['price'] = df2['price'].astype(str)

    # clean data
    pd.options.mode.chained_assignment = None  # default='warn'
    df2['host_acceptance_rate'] = df2['host_acceptance_rate'].str.replace("%", "").astype("float")
    df2['price'] = df2['price'].str.replace("[$, ]", "").astype("float")
    df2['host_response_rate'] = df2['host_response_rate'].str.replace("%", "").astype("float")

    df2['superhost']=np.where(df2['host_is_superhost']=='t',1,0)
    del df2['host_is_superhost']


    # select non-numeric variables and create dummies
    non_num_vars = df2.select_dtypes(include=['object']).columns
    dummy_vars = pd.get_dummies(df2[non_num_vars])

    # drop non-numeric variables from df2 and add the dummies
    df3=df2.drop(non_num_vars,axis=1)
    df3 = pd.merge(df3,dummy_vars, left_index=True, right_index=True)

    #Droping the missing values

    df3=df3.dropna()
    return df3

In [0]:
def loadData(filename):
    listings=pd.read_csv(filename)
    return listings


In [0]:
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from math import sqrt

target_variable = ["price"]

def train_model_ridge(dataframe):
    model_object = {}
    print ("Training Ridge Regression")
    print (dataframe.columns.tolist())
    X_train, X_test, y_train, y_test = train_test_split(dataframe.drop(target_variable, axis=1),
                                                        dataframe.price)
    model = linear_model.LinearRegression()
    model.fit(X_train, y_train)
    y_test_pred=model.predict(X_test)
    y_train_pred=model.predict(X_train)
    rms_test=sqrt(mean_squared_error(y_test,y_test_pred))
    mean_squared_train=mean_squared_error(y_train, y_train_pred)
    mean_squared_test=mean_squared_error(y_test, y_test_pred)
    r2_score_train=(r2_score(y_train, y_train_pred))
    r2_score_test=r2_score(y_test, y_test_pred)
    print('RMSE',rms_test)
    
    return mean_squared_train,mean_squared_test,r2_score_train,r2_score_test,rms_test,model

In [0]:
def train_random_forest(dataframe):
    X_train, X_test, y_train, y_test = train_test_split(dataframe.drop(target_variable, axis=1),
                                                        dataframe.price)
    print ("Training Random Forest")
    rf = RandomForestRegressor(n_estimators=100, criterion='mse', n_jobs=5,verbose=2)
    rf.fit(X_train, y_train)
    y_test_pred=rf.predict(X_test)
    y_train_pred=rf.predict(X_train)
    rms_test=sqrt(mean_squared_error(y_test,y_test_pred))
    mean_squared_train=mean_squared_error(y_train, y_train_pred)
    mean_squared_test=mean_squared_error(y_test, y_test_pred)
    r2_score_train=(r2_score(y_train, y_train_pred))
    r2_score_test=r2_score(y_test, y_test_pred)
    print('RMSE',rms_test)
    
    return mean_squared_train,mean_squared_test,r2_score_train,r2_score_test,rms_test,rf

In [0]:
from dask.multiprocessing import get
dsk={'Task0':(loadData,'/content/listings.csv'),
     'Task1':(numeric_conversion,'Task0'),
     'Task2':(missing_value,'Task1'),
     'Task3':(feature,'Task2'),
     'Task4': (train_model_ridge,'Task3'),
     'Task5': (train_random_forest ,'Task3')}

In [0]:
get(dsk,'Task4')

Training Ridge Regression
['host_response_rate', 'host_acceptance_rate', 'security_deposit', 'cleaning_fee', 'extra_people', 'host_listings_count', 'zipcode', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'number_of_reviews', 'review_scores_rating', 'reviews_per_month', 'superhost', 'property_type_Apartment', 'property_type_Bed & Breakfast', 'property_type_Boat', 'property_type_Bungalow', 'property_type_Cabin', 'property_type_Camper/RV', 'property_type_Chalet', 'property_type_Condominium', 'property_type_Dorm', 'property_type_House', 'property_type_Loft', 'property_type_Other', 'property_type_Tent', 'property_type_Townhouse', 'property_type_Treehouse', 'property_type_Yurt', 'room_type_Entire home/apt', 'room_type_Private room', 'room_type_Shared room', 'cancellation_policy_flexible', 'cancellation_policy_moderate', 'cancellation_policy_strict']
RMSE 45.66469121350048


(2593.5425550740347,
 2085.2640236243483,
 0.6301652364921195,
 0.6210244563347187,
 45.66469121350048,
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False))

In [0]:
get(dsk,'Task5')

Training Random Forest


[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.


building tree 1 of 100
building tree 4 of 100
building tree 3 of 100
building tree 2 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 10 of 100
building tree 11 of 100
building tree 9 of 100
building tree 12 of 100
building tree 14 of 100
building tree 13 of 100
building tree 16 of 100
building tree 15 of 100
building tree 17 of 100
building tree 19 of 100
building tree 18 of 100
building tree 20 of 100
building tree 21 of 100
building tree 23 of 100
building tree 22 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 28 of 100
building tree 27 of 100
building tree 29 of 100
building tree 30 of 100
building tree 32 of 100
building tree 33 of 100
building tree 31 of 100
building tree 34 of 100
building tree 36 of 100


[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    0.4s


building tree 35 of 100
building tree 39 of 100
building tree 37 of 100
building tree 38 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 45 of 100
building tree 43 of 100
building tree 46 of 100
building tree 44 of 100
building tree 47 of 100
building tree 48 of 100
building tree 51 of 100
building tree 50 of 100
building tree 49 of 100
building tree 52 of 100
building tree 53 of 100
building tree 55 of 100
building tree 54 of 100
building tree 56 of 100
building tree 58 of 100
building tree 57 of 100
building tree 59 of 100
building tree 61 of 100
building tree 60 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 69 of 100
building tree 68 of 100
building tree 70 of 100
building tree 71 of 100
building tree 73 of 100
building tree 72 of 100
building tree 74 of 100
building tree 76 of 100
building tree 79 of 100
building tree 75

[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    1.3s finished
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/base.py:154: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/base.py:154: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


RMSE 43.17238237054093


(365.55987674535413,
 1863.8545995481932,
 0.9475304023053132,
 0.6698196063566645,
 43.17238237054093,
 RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=5,
                       oob_score=False, random_state=None, verbose=2,
                       warm_start=False))

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  6 16:03:50 2019

@author: Tripti Santani
"""

from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

target_variable = ["price"]

def train_model_ridge(dataframe):
    model_object = {}
    print ("Training Ridge Regression")
    print (dataframe.columns.tolist())
    ridge = linear_model.Ridge(alpha=0.5)
    X_train, X_test, y_train, y_test = train_test_split(dataframe.drop(target_variable, axis=1),
                                                        dataframe.Sales)
    model = ridge.fit(X_train, y_train)
    model.fit(X_train, y_train)
    model_object["model"] = model
    model_object["training_features"] = X_train.columns.tolist()
    return (model_object)

def train_model_with_grid_search(dataframe):
    print ("Training Ridge Regression")
    print (dataframe.columns.tolist())
    ridge = linear_model.Ridge()
    params_grid = {
        "alpha": [0.01, 0.05, 0.1, 0.5]
    }
    X_train, X_test, y_train, y_test = train_test_split(dataframe.drop(target_variable, axis=1),
                                                        dataframe.Sales)
    model = GridSearchCV(ridge, param_grid=params_grid, verbose=2, cv=5, refit=True, n_jobs=5)
    model.fit(X_train, y_train)
    return model.best_estimator_


def train_random_forest(dataframe):
    X_train, X_test, y_train, y_test = train_test_split(dataframe.drop(target_variable, axis=1),
                                                        dataframe.Sales)
    print ("Training Random Forest")
    rf = RandomForestRegressor(n_estimators=100, criterion='mse', n_jobs=5,verbose=2)
    rf.fit(X_train, y_train)
    return rf

def test_model_ridge(dataframe,model):
    df = dataframe.drop(target_variable, axis=1)
    return model.predict(df)

if __name__ == '__main__':
    pass